## Part I 
## Raw Data Process, Word2Vec and Sentiment Analysis

In [82]:
import pandas as pd
import numpy as np

### Step 1) Read Twitter csv file and compress the data to one-line-each-day format (concatenate all tweets within one day to a single string)

In [83]:
# Read twitter csv file
tw = pd.read_csv('corona.csv', encoding = "ISO-8859-1")
tw

,date,time,tweet
0,1/1/20,16:05:54,El M*gnici*io es la Õ_nica opciÕ_n para Venezu...
1,1/1/20,16:05:03,"plying to @ironorehopper\rInteresting, further..."
2,1/1/20,15:41:02,"Meu Deus, tÕ£o inventando nome pra tudo ""White..."
3,1/1/20,14:59:53,"Yo a ti conozco en persona , y eres el q se ga..."
4,1/1/20,14:42:13,"Soy balsa , es mi saludo y yo saludo a quien q..."
...,...,...,...
381679,1/31/20,18:49:43,#Coronavirus https://twitter.com/OSWALDORIOSM...
381680,1/31/20,18:49:43,Coronavirus: What are the symptoms? https://w...
381681,1/31/20,18:49:43,Chicago PMIs > coronavirus
381682,1/31/20,18:49:43,"Como bolsonaro Ì© serviÌ¤al americano, brasile..."


In [315]:
# Get unique date values in order to locate the tweets within same day
dates = tw.date
dates_unique = set(dates)
dates_unique

{'1/1/20',
 '1/10/20',
 '1/11/20',
 '1/12/20',
 '1/13/20',
 '1/14/20',
 '1/15/20',
 '1/16/20',
 '1/17/20',
 '1/18/20',
 '1/19/20',
 '1/2/20',
 '1/20/20',
 '1/21/20',
 '1/22/20',
 '1/23/20',
 '1/24/20',
 '1/25/20',
 '1/26/20',
 '1/27/20',
 '1/28/20',
 '1/29/20',
 '1/3/20',
 '1/30/20',
 '1/31/20',
 '1/4/20',
 '1/5/20',
 '1/6/20',
 '1/7/20',
 '1/8/20',
 '1/9/20',
 '2/1/20',
 '2/10/20',
 '2/11/20',
 '2/12/20',
 '2/13/20',
 '2/14/20',
 '2/15/20',
 '2/16/20',
 '2/17/20',
 '2/18/20',
 '2/19/20',
 '2/2/20',
 '2/20/20',
 '2/21/20',
 '2/22/20',
 '2/23/20',
 '2/24/20',
 '2/25/20',
 '2/26/20',
 '2/27/20',
 '2/28/20',
 '2/29/20',
 '2/3/20',
 '2/4/20',
 '2/5/20',
 '2/6/20',
 '2/7/20',
 '2/8/20',
 '2/9/20',
 '3/1/20',
 '3/10/20',
 '3/11/20',
 '3/12/20',
 '3/13/20',
 '3/14/20',
 '3/15/20',
 '3/16/20',
 '3/17/20',
 '3/18/20',
 '3/19/20',
 '3/2/20',
 '3/20/20',
 '3/21/20',
 '3/22/20',
 '3/23/20',
 '3/24/20',
 '3/25/20',
 '3/26/20',
 '3/27/20',
 '3/28/20',
 '3/29/20',
 '3/3/20',
 '3/30/20',
 '3/31/20',
 

In [85]:
compressed = pd.DataFrame({'dates':[], 'tweets':[]})
for i in dates_unique:         # for each date: 
    df = tw[tw['date']==i]     # get a dataframe containing all tweets in that day
    twt_combined = ''          # make up a string to store tweet contents
    for j in df['tweet']:
        twt_combined = twt_combined+j        
        twt_combined = twt_combined+'. \n'
    df_compressed = pd.DataFrame({'dates':[i], 'tweets':[twt_combined]})
    compressed = pd.concat([compressed, df_compressed])
compressed['dates'] = pd.to_datetime(compressed['dates'])
compressed.sort_values(by = 'dates', inplace = True)
compressed.set_index('dates', inplace = True)
compressed

,tweets
dates,
2020-01-01,El M*gnici*io es la Õ_nica opciÕ_n para Venezu...
2020-01-02,"Sim, mas nas redes sociais ninguÕ©m precisa pe..."
2020-01-03,A pan-coronavirus fusion inhibitor targeting t...
2020-01-04,Estados Unidos pide a Rep. Dom. que le mande a...
2020-01-05,TÕ no papo kkkkskskks. \nEnero el mes de las ...
...,...
2020-04-21,CoronavÕ_rus: Brasil registra primeira morte d...
2020-04-22,#Coronavirus: un colegio de Laboulaye le deja ...
2020-04-23,With the scarcity of #foodservice jobs availab...


### Step 2) Use Word2Vec model to train WSJ files and get coronavirus related library

In [133]:
import os
import collections
from collections import Counter
from gensim.models import Word2Vec #to install gensim, open powershell prompt, and type in: conda install -c anaconda gensim
from gensim.test.utils import get_tmpfile
import csv     

class MySentences(object):      #this function helps us read all of the txt files in the designated folder
    def __init__(self, dirname):
        self.dirname = dirname

    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname),encoding = "ISO-8859-1"):
                yield line.split("', '")

def w2v(text):      #this function trains the word2vector model (output) using text documents (input)
    model_w2v = Word2Vec(text)
    return model_w2v

def Convert(lst):     #this function converts a list of list (input) into a dictionary (output)
    it = iter(lst)
    res_dct = dict(it)
    return res_dct

def find_similar(word,model_w2v):   #this function returns the expanded dictionary (most similar words to the seed words)
    word_dict = model_w2v.wv.most_similar(positive=[word],topn=20)   
    d = Convert(word_dict)
    return d

def get_seedword(keyword,dict):
    return dict.get(keyword)


def freq(model):
    freq_dict = {}
    for word in model.wv.vocab:
        freq_dict.update({word: model.wv.vocab[word].count})
    return freq_dict

def main():
    curDir = os.getcwd()  
    os.chdir('/Users/liwenqian/Documents') 
    sentences = MySentences('./wsj')
    model_w2v = w2v(sentences)
    model_w2v.save("word2vec.model")
    freq_dict = freq(model_w2v)
    
    sorted_x = sorted(freq_dict.items(), key=lambda kv: kv[1],reverse=True)
    sorted_dict = collections.OrderedDict(sorted_x)
    with open("frequencies.csv", "w") as csv_file:
        csvwriter = csv.writer(csv_file)
        for item in sorted_dict:
            csvwriter.writerow([item,sorted_dict[item]])


    #seed words:
    category = {'virus': ['virus','wuhan','covid','sick','coronavirus'],
                'pandemic': ['outbreak','transmiss','symptom','remdesivir'],
                'test': ['test','index','stay'],
                'mask': ['mask','droplet','flatten'],
                'health': ['health','cure'],
                'social': ['social','contract','law'],
                'market': ['market','uncertain','risk','plummet','circuit']}
    
    keywords = []
    for item in category.keys():
        keywords.append(item)
    keyword_dict = {}
    for keyword in keywords:
        seedword_dict = {}
        seedwords = get_seedword(keyword,category)
        for seedword in seedwords:
            sim_dict = find_similar(seedword,model_w2v)
            seedword_dict.update({seedword: sim_dict})
        keyword_dict.update({keyword: seedword_dict})

    #Remove all the repeated words that do not have the highest probability
    update_dict = dict(keyword_dict) #copy dictionary from above
    for keyword in keyword_dict.keys():
        lst = [] #create list of words for the whole dictionary
        for seedword in keyword_dict[keyword]:
            for item in keyword_dict[keyword][seedword]:
                lst.append(item)
        cnt = Counter(lst) #count the words frequencies
        repeat_item = [x for x, y in cnt.items() if y > 1] #find repeated words
        print(repeat_item)
        for word in repeat_item:
            repeat_dict = {} #create probability dictionary for each word in the form {probability1:word,probability2:word...}
            for seedword in keyword_dict[keyword]:
                if word in list(keyword_dict[keyword][seedword].keys()):
                    repeat_dict.update({keyword_dict[keyword][seedword][word]: word})
            prob = [key for key in repeat_dict.keys()] #get probabilities of all repeated words
            max_prob = max(prob)
            for seedword in keyword_dict[keyword]:
                if word in list(keyword_dict[keyword][seedword].keys()):
                    if max_prob and keyword_dict[keyword][seedword][word] != max_prob:
                        del update_dict[keyword][seedword][word] #delete the words that do not have maximum probability

    with open('updated_dict.csv', 'w') as csv_file:
        csvwriter = csv.writer(csv_file, delimiter='\t')
        for keyword in update_dict:
            for seedword in update_dict[keyword]:
                csvwriter.writerow([keyword, ',', seedword, ',', list(update_dict[keyword][seedword].keys())])


if __name__ == "__main__":
    main()

['rapid', 'viral', 'covid', 'coronavirus', 'slow', 'asymptomat', 'flu', 'sar', 'ebola', 'influenza', 'epicent', 'summari', 'virus']
['sar', 'dead', 'contagion', 'respiratori']
[]
[]
[]
[]
[]


In [145]:
# Clean the words in library because some words are with characters like []'
updated_dict = pd.read_csv('updated_dict.csv')
lib = []
for i in updated_dict:
    vec = updated_dict[i]
    vec = vec.dropna()
    for j in vec:
        signs = "'[] "
        for k in signs:
            j = j.replace(k,'')
        lib.append(j)
lib   

['highest',
 'ill',
 'selfisol',
 'stem',
 'toll',
 'sar',
 'pneumonia',
 'antivir',
 'sampl',
 'seng',
 'stuck',
 'glove',
 'approx',
 'though',
 'mount',
 'persist',
 'physic',
 'signific',
 'regul',
 'bond',
 'grief',
 'transmiss',
 'yuan',
 'cathedr',
 'spain',
 'flu',
 'leav',
 'orlean',
 'pandem',
 'pathogen',
 'suspect',
 'antimalari',
 'posit',
 'yield',
 'famili',
 'face',
 'cling',
 'uncheck',
 'emerg',
 'lifethreaten',
 'recommend',
 'increas',
 'assist',
 'profit',
 'prospect',
 'serious',
 'slid',
 'coowner',
 'provinc',
 'influenza',
 'worri',
 'handl',
 'spread',
 'relat',
 'expos',
 'arthriti',
 'symptom',
 'sale',
 'work',
 'gown',
 'gust',
 'unknow',
 'intens',
 'complic',
 'distanc',
 'ravag',
 'guidanc',
 'forecast',
 'starter',
 'becom',
 'fastest',
 'tick',
 'deadliest',
 'mild',
 'someon',
 'escal',
 'itali',
 'bat',
 'die',
 'antiinflammatori',
 'identifi',
 'profit',
 'selfisol',
 'goggl',
 'aerosol',
 'catch',
 'barton',
 'recurr',
 'approx',
 'hemorrhag',
 'g

### Step 3) Filter tweets for relevant contexts. For sentences with words in the library, keep that sentence. Otherwise drop that sentence.

In [253]:
relevant_tweets = []
signs = "`~1234567890-=!@#$%^&*()_+[]\{}|;':,./<>?"
for tweet in compressed['tweets']:        # for each day's tweet:
    sentences = tweet.split('\n')         # break into sentences,
    tweet_new=[]
    for i in sentences:                   # for each sentence:
        i_spl = i.split(' ')              # break into tokens,
        toks = []
        for j in i_spl:
            for k in signs:
                j = j.replace(k, '')
            toks.append(j)  
    #     print(toks)
        for m in toks:                    # iterate over tokens in that sentence:
            m_stem = stemmer.stem(m)      
            if m_stem in lib or m in lib: # once encounter a token or its stem that is in the lib,
                tweet_new.append(i)       # keep that sentence, stop iterating that sentence,
                break                     # and iterate next sentence.
    new_tweet=''
    for sent in tweet_new:                # concatenate tweets within same day into one single string
        new_tweet = new_tweet + '\n' + sent
    new_tweet = new_tweet.strip()
    relevant_tweets.append(new_tweet)     # get relevant tweets for each day and make into a data frame

compressed_relevant = pd.DataFrame({'tweets':relevant_tweets}, index = compressed.index)

In [261]:
compressed_relevant

,tweets
dates,
2020-01-01,"plying to @ironorehopper\rInteresting, further..."
2020-01-02,"Ten que corijir mesmo, odÕ_io cuando mim ler a..."
2020-01-03,A pan-coronavirus fusion inhibitor targeting t...
2020-01-04,Tengo imÕgenes que dicen lo contrario.. \nSur...
2020-01-05,Supposedly all the usual suspects ruled out. P...
...,...
2020-04-21,CUANDO QUIERAS CORONAVIRUS PEDORRO!!! https:/...
2020-04-22,Are you out of work due to #coronavirus? Is yo...
2020-04-23,"If the coronavirus response team were a band, ..."


### Step 4) Perform daily sentiment analysis and make record. 

In [264]:
# Define all relevant functions to be used for sentiment analysis
import re    
import os    
import os.path
import string
import nltk
from nltk.tokenize import MWETokenizer  #import tokenizer
nltk.download('stopwords')  #download the list of stopwords, if you have not already done so
from nltk.corpus import stopwords  #import the list of stopwords
from nltk.stem.snowball import SnowballStemmer  #import stemmer module
import pandas as pd

# Negation handling
stemmer = SnowballStemmer('english')
def negation_handling (text):
    negatereg1=re.compile(r'n’t \S*')
    negatereg2=re.compile(r'no \S*')
    negatereg3=re.compile(r'not \S*')
    
    negate_list1=negatereg1.findall(docu)
    negate_list2=negatereg2.findall(docu)
    negate_list3=negatereg3.findall(docu)
    negate_list = negate_list1 + negate_list2 + negate_list3

    text1 = re.sub(negatereg1,'', text) 
    text2 = re.sub(negatereg2,'', text1) 
    text3 = re.sub(negatereg3,'', text2) 
    
    return text3, negate_list

# remove punctuations, remove stopwords, stem and tokenize
tokenizer = MWETokenizer([('new', 'york'), ('san', 'francisco')])
stemmer = SnowballStemmer('english')
def clean_tokenize (text):
    # remove numbers
    text_nonum = re.sub(r'\d+', '', text)
    text_nonum = text_nonum.replace('”', '')
    text_nonum = text_nonum.replace('“', '')
    text_nonum = text_nonum.replace('—', ' ')
    # remove punctuations and convert characters to lower case
    text_nopunct = "".join([char.lower() for char in text_nonum if char not in string.punctuation]) 
    # substitute multiple whitespace with single whitespace. Also, removes leading and trailing whitespaces
    text_cleaned = re.sub('\s+', ' ', text_nopunct).strip()
    text_output = tokenizer.tokenize(text_cleaned.split())
    text_stopwords = []
    for word in text_output:
        if word not in stopwords.words('english'):  #filter the stop words
            text_stopwords.append(word) 
    text_stemmed = ([stemmer.stem(w) for w in text_stopwords])
    return text_stemmed

# sentiment classification and construction of sentiment measures
df1 = pd.read_excel('LM Sentiment Dictionary.xlsx', sheet_name='Positive')
positive_temp = df1['WORD'].tolist()
positive = [item.lower() for item in positive_temp]
positive_stemmed = ([stemmer.stem(w) for w in positive])
positive_negate = []
for word in positive_stemmed:
    positive_negate.append('n’t '+ word)
    positive_negate.append('no '+ word)
    positive_negate.append('not '+ word)

df2 = pd.read_excel('LM Sentiment Dictionary.xlsx', sheet_name='Negative')  
negative_temp = df2['WORD'].tolist()
negative = [item.lower() for item in negative_temp]
negative_stemmed = ([stemmer.stem(w) for w in negative])
negative_negate = []
for word in negative_stemmed:
    negative_negate.append('n’t '+ word)
    negative_negate.append('no '+ word)
    negative_negate.append('not '+ word)

positive_final = positive_stemmed + negative_negate
negative_final = negative_stemmed + positive_negate


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/liwenqian/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [282]:
dates_relevant = pd.Series(compressed_relevant.index)
sentiments = []
for i in range(len(compressed_relevant)):
    docu = compressed_relevant['tweets'][i]

    docu_negate, nlist = negation_handling(docu)
    nlist_stemmed = ([stemmer.stem(w) for w in nlist])

    docu_tokens = clean_tokenize(docu_negate)

    docufinal = docu_tokens + nlist_stemmed

    doculen = len(docufinal)
    docu_positive = ([w for w in docufinal if w in positive_final])
    docu_negative = ([w for w in docufinal if w in negative_final])
    docu_sentiment = (len(docu_positive)-len(docu_negative))/doculen
    
    sentiments.append(docu_sentiment)
    print(dates_relevant[i], docu_sentiment)

2020-01-01 00:00:00 -0.013623978201634877
2020-01-02 00:00:00 -0.013513513513513514
2020-01-03 00:00:00 -0.012
2020-01-04 00:00:00 -0.002638522427440633
2020-01-05 00:00:00 -0.01
2020-01-06 00:00:00 -0.017761989342806393
2020-01-07 00:00:00 -0.009259259259259259
2020-01-08 00:00:00 -0.03417935702199661
2020-01-09 00:00:00 -0.035932138772398016
2020-01-10 00:00:00 -0.02071192313750197
2020-01-11 00:00:00 -0.015017956252040483
2020-01-12 00:00:00 -0.015471167369901548
2020-01-13 00:00:00 -0.013071895424836602
2020-01-14 00:00:00 -0.029275733123400906
2020-01-15 00:00:00 -0.01881917020135537
2020-01-16 00:00:00 -0.028658691779078645
2020-01-17 00:00:00 -0.02727640995633974
2020-01-18 00:00:00 -0.026542831549667627
2020-01-19 00:00:00 -0.027964244721922715
2020-01-20 00:00:00 -0.0163265306122449
2020-01-21 00:00:00 -0.01838134947152444
2020-01-22 00:00:00 -0.024015513126491646
2020-01-23 00:00:00 -0.02865118069815195
2020-01-24 00:00:00 -0.0223209068598808
2020-01-25 00:00:00 -0.0238517485

In [312]:
# Convert the result to a data frame and output to a csv file.
sentiment_array = pd.DataFrame({'sentiments':sentiments},index = dates_relevant)
sentiment_array.to_csv('Sentiments_Coronavirus.csv')
sentiment_array

,sentiments
dates,
2020-01-01,-0.013624
2020-01-02,-0.013514
2020-01-03,-0.012000
2020-01-04,-0.002639
2020-01-05,-0.010000
...,...
2020-04-21,-0.026963
2020-04-22,-0.026032
2020-04-23,-0.019547
